In [2]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
from sklearn.model_selection import train_test_split

import keras
import random
from keras.datasets import mnist
from keras.models import Model
from keras.models import Sequential
from keras import regularizers
from keras import metrics

from keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, Activation, BatchNormalization, Concatenate
from keras.layers import MaxPooling1D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from keras import backend as K
from matplotlib import pyplot as plt

from sklearn.neighbors import DistanceMetric

Using TensorFlow backend.


In [3]:
import pickle
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

In [4]:
# vggish_feature_dir = '/mnt/tmp/SED_Studies/train1_features.txt'
# vggish_feature_dir = '/mnt/tmp/SED_Studies/train1_features_5000.txt'
vggish_feature_dir = '/mnt/tmp/SED_Studies/train1_features_12000-2-5.txt'
pickle_in = open(vggish_feature_dir,"rb")
vggish_feature_label_file = pickle.load(pickle_in)

In [5]:
print(len(vggish_feature_label_file))
print(len(vggish_feature_label_file[0]))       ## number of item in one line 
print(type(vggish_feature_label_file[0][0]))   ## the first item is an array that saves 892 dimensional features 
print(vggish_feature_label_file[0][0].shape)   ## the first item is an array that saves 892 dimensional features 
print(len(vggish_feature_label_file[0][1]))    ## the second item is a list that strings of lables 
print(vggish_feature_label_file[0][1])         ## the second item is a list that strings of lables 
print(vggish_feature_label_file[0][2])         ## the third item is a string that saves the directory of the audio file

12000
3
<class 'numpy.ndarray'>
(19, 128)
2
['drilling', 'jackhammer']
/home/t-anmend/UrbanSound8K/soundscapes/train_12000/2_5/uniform/soundscape_train_1_uniform0.wav


In [6]:
# for i in range(0, 100):
#     print(vggish_feature_label_file[i][1])

In [7]:
# for i in range(0, 100):
#     print(All_Labels_One_Hot[i])

In [8]:
All_Features = []
for i in range(0, len(vggish_feature_label_file)):
    All_Features.append(vggish_feature_label_file[i][0])

print(len(All_Features))
print(All_Features[0].shape)
# print(All_Features_Mean_Std[0])

12000
(19, 128)


In [9]:
for i in range(len(All_Features)):
    All_Features[i] = All_Features[i].astype('float32')
    All_Features[i] /= 255
# print(All_Features_Mean_Std[0])

In [10]:
All_Labels = []
for i in range(0, len(vggish_feature_label_file)):
    All_Labels.append(vggish_feature_label_file[i][1])
    
one_hot = MultiLabelBinarizer()
# One-hot encode data
All_Labels_One_Hot = one_hot.fit_transform(All_Labels)
print(All_Labels_One_Hot.shape)
print(All_Labels_One_Hot[0])

(12000, 10)
[0 0 0 0 1 0 0 1 0 0]


In [11]:
num_files = len(All_Features)
num_time_steps = All_Features[0].shape[0]
num_features = All_Features[0].shape[1]
print(num_files)
print(num_time_steps)
print(num_features)

12000
19
128


In [12]:
min_feature = np.ones(128)
max_feature = np.zeros(128)

def getMinimumm():
    num_lines = len(All_Features)
    for feature_id in range(0, num_features):
        for file_id in range(0, num_files):
#             minElement = np.amin(feature_array_one_file[:, col])
#         print('min', minElement)
            for line_id in range(0, num_time_steps):
                minElement = All_Features[file_id][line_id][feature_id]
                if(minElement < min_feature[feature_id]):
                    min_feature[feature_id] = minElement
        
def getMaximumm():
    num_lines = len(All_Features)
    for feature_id in range(0, num_features):
        for file_id in range(0, num_files):
#             minElement = np.amin(feature_array_one_file[:, col])
#         print('min', minElement)
            for line_id in range(0, num_time_steps):
                maxElement = All_Features[file_id][line_id][feature_id]
#             maxElement = np.amax(feature_array_one_file[:, col])
#         print('max', maxElement)
                if(maxElement > max_feature[feature_id]):
                    max_feature[feature_id] = maxElement 

getMinimumm()
getMaximumm()

In [13]:
Normalized_All_Features = []
difference = max_feature - min_feature
for file_id in range(0, num_files):
#     one_file_feature = np.zeros(128)
    one_file_feature = np.zeros((19, 128))
    
    for line_id in range(0, num_time_steps):
        for feature_id in range(0, num_features):
            one_file_feature[line_id][feature_id] = (All_Features[file_id][line_id][feature_id] - min_feature[feature_id])/difference[feature_id]       
    Normalized_All_Features.append(one_file_feature)    
        
print(len(Normalized_All_Features))
print(Normalized_All_Features[0].shape)
# print(Normalized_All_Features_Mean_Std[0])
# print(difference)


12000
(19, 128)


In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(Normalized_All_Features, All_Labels_One_Hot, test_size=0.20, random_state=42)

print(len(X_train))
print(len(X_test))
print(len(Y_train))
print(len(Y_test))

9600
2400
9600
2400


In [15]:
###### This is  distance (it sums up each element) so that the output layer is 1 node 
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    print('shape1[0]')
    print(shape1[0])
    return (shape1[0], 1)

In [16]:
# distance measure
###### This is not actually distance (it didn't sum up each element) so that the output layer is not 1 node 
def cosine_distance(vecs, normalize=False):
    x, y = vecs
    if normalize:
        x = K.l2_normalize(x, axis=0)
        y = K.l2_normalize(x, axis=0)
    return K.prod(K.stack([x, y], axis=1), axis=1)

def cos_dist_output_shape(shapes):
#     print('shapes[0]')
#     print(shapes[0])
    return shapes[0]

In [17]:
# time_steps = 10
# freq_bins = 128
# classes_num = 527
time_steps = 19
freq_bins = 128
# time_steps = 1
# freq_bins = 256
# num_classes = 10
num_labels = 10

# Hyper parameters
hidden_units = 128
drop_rate = 0.5
batch_size = 32
# epochs  = 10
epochs  = 1

In [18]:
def create_base_network():  
    input_layer = Input(shape=(time_steps, freq_bins))

    a1 = Dense(hidden_units)(input_layer)
    a1 = BatchNormalization()(a1)
    a1 = Activation('relu')(a1)
    a1 = Dropout(drop_rate)(a1)

    a2 = Dense(hidden_units)(a1)
    a2 = BatchNormalization()(a2)
    a2 = Activation('relu')(a2)
    a2 = Dropout(drop_rate)(a2)

    a3 = Dense(hidden_units)(a2)
    a3 = BatchNormalization()(a3)
    a3 = Activation('relu')(a3)
    a3 = Dropout(drop_rate)(a3)
    output_layer = Dense(128, activation='relu')(a3)

    model = Model(inputs=input_layer, outputs=output_layer)

    return model

In [19]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
#     print(y_true)
#     print(y_pred)
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)
## in this case, if y_true = 1, it is recognized correct if y_pred is 0;  
## if y_true = 0, it is recognized correct if y_pred is 1 

In [20]:
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [21]:
# img_rows, img_cols = 28, 28
# # the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
# #     input_shape = (img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255

# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')
# # input_shape = x_train.shape[1:]
# # print(input_shape)

In [22]:
# x_train = np.squeeze(x_train, axis=3)
# x_test = np.squeeze(x_test, axis=3)
# print('x_train shape:', x_train.shape)
# print('x_test shape:', x_test.shape)

In [23]:
def get_y_xor(y1, y2):
    y_diff = np.zeros(num_labels)
    for i in range(num_labels):
        if y1[i] == y2[i]:
            y_diff[i] = 1
        else :
            y_diff[i] = 0
    return y_diff

In [24]:
def get_four_conditions(y1, y2):
    y_diff = np.zeros(num_labels * 3)
#     for i in range(num_labels):
#         if y1[i] == y2[i] and y1[i] == 1:
#             y_diff[i] = 1
#         else :
#             y_diff[i] = 0
#     return y_diff

    for i in range(num_labels):
#         print(y1)
#         print(y2)
        if y1[i] == y2[i] and y1[i] == 1:
            y_diff[i*3 + 0] = 1
        elif y1[i] == y2[i] and y1[i] == 0:
            y_diff[i*3 + 1] = 1
        elif y1[i] != y2[i] and y1[i] == 1:
            y_diff[i*3 + 2] = 1
        elif y1[i] != y2[i] and y1[i] == 0:
            y_diff[i*3 + 2] = 1


#     y_diff = np.zeros(4)
#     if y1 == y2 and y1 == 1:
#         y_diff[0] = 1
#     elif y1 == y2 and y1 == 0:
#         y_diff[1] = 1
#     elif y1 != y2 and y1 == 1:
#         y_diff[2] = 1
#     elif y1 != y2 and y1 == 0:
#         y_diff[3] = 1

    return y_diff

In [25]:
def checkIfExist(seen, z1, z2):
#     string_id1 = [(str(z1) + "_" + str(z2))]
#     string_id2 = [(str(z2) + "_" + str(z1))]
    string_id1 = (str(z1) + "_" + str(z2))
    string_id2 = (str(z2) + "_" + str(z1))
    exist = False
    if string_id1 not in seen or string_id2 not in seen:
        seen.add(string_id1)
#         part1 = string_id.split('_')[0]
#         part2 = string_id.split('_')[1]
#         reversed_id = part2 + "_" + part1
#         seen.add(reversed_id)
        seen.add(string_id2)
    else:
        exist = True
    return seen, exist

In [26]:
# def create_pairs(x, y, digit_indices_one, digit_indices_zero, offset):
#     '''Positive and negative pair creation.
#     Alternates between positive and negative pairs.
#     '''
#     pairs = []
#     labels = []
#     ids = []
# #     n = min([len(digit_indices[d]) for d in range(num_labels)]) - 1
# #     print('n is ', n)
#     for d in range(num_labels):
        
# #         min_index = min(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0]) - offset
# #         max_index = max(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0])
        
#         for k in range(1, offset):
#             min_index = min(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0]) - k
#             max_index = max(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0]) - k
            
#             for i in range(min_index):
#                 second_index = i + k
                
#                 z1, z2 = digit_indices_one[d][i], digit_indices_one[d][i + k]    ###  same pair  one one 
#                 pairs += [[x[z1], x[z2]]]
#                 y_diff = get_four_conditions(y[z1], y[z2])
# #                 y_diff = get_two_conditions(y[z1], y[z2])
# #                 y_diff = get_y_xor(y[z1], y[z2])
# #                 y_diff = get_hamming_dis(y[z1], y[z2])
#                 labels += [y_diff]
#                 ids += [(str(z1) + "_" + str(z2))]
# #                 print("same pair11  ", i , "  ", second_index)
# #                 print("same pair11  ", z1 , "  ", z2)
                
#                 z1, z2 = digit_indices_zero[d][i], digit_indices_zero[d][i + k]  ###  same pair   zero zero 
#                 pairs += [[x[z1], x[z2]]]                       
#                 y_diff = get_four_conditions(y[z1], y[z2])
# #                 y_diff = get_two_conditions(y[z1], y[z2])
# #                 y_diff = get_y_xor(y[z1], y[z2])
# #                 y_diff = get_hamming_dis(y[z1], y[z2])
#                 labels += [y_diff]
#                 ids += [(str(z1) + "_" + str(z2))]
# #                 print("same pair00  ", i , "  ", second_index)
# #                print("same pair00  ", z1 , "  ", z2)

#                 z1, z2 = digit_indices_one[d][i], digit_indices_zero[d][i + k]       ###  not same pair  one zero 
#                 pairs += [[x[z1], x[z2]]]
#                 y_diff = get_four_conditions(y[z1], y[z2])
# #                 y_diff = get_two_conditions(y[z1], y[z2])
# #                 y_diff = get_y_xor(y[z1], y[z2])
# #                 y_diff = get_hamming_dis(y[z1], y[z2])
#                 labels += [y_diff]
#                 ids += [(str(z1) + "_" + str(z2))]
# #                 print("not same pair10  ", i , "  ", second_index)
# #                print("not same pair10  ", z1 , "  ", z2)
                
#                 z1, z2 = digit_indices_one[d][i + k], digit_indices_zero[d][i]  ###  not same pair   zero one 
#                 pairs += [[x[z1], x[z2]]]
#                 y_diff = get_four_conditions(y[z1], y[z2])
# #                 y_diff = get_two_conditions(y[z1], y[z2])
# #                 y_diff = get_y_xor(y[z1], y[z2])
# #                 y_diff = get_hamming_dis(y[z1], y[z2])
#                 labels += [y_diff]
#                 ids += [(str(z1) + "_" + str(z2))]
# #                 print("not same pair01  ", second_index , "  ", i)
# #                print("not same pair01  ", z1 , "  ", z2)
            
#             for i in range(min_index, max_index):
                
#                 second_index = i + k
                
#                 if digit_indices_one[d].shape[0] > digit_indices_zero[d].shape[0]:    ##   if there are more 1 than 0 
#                     z1, z2 = digit_indices_one[d][i], digit_indices_one[d][i + k]     ###  same pair  one one 
#                     pairs += [[x[z1], x[z2]]]                      
#                     y_diff = get_four_conditions(y[z1], y[z2])
# #                 y_diff = get_two_conditions(y[z1], y[z2])
# #                 y_diff = get_y_xor(y[z1], y[z2])
# #                 y_diff = get_hamming_dis(y[z1], y[z2])
#                     labels += [y_diff]
#                     ids += [(str(z1) + "_" + str(z2))]
# #                     print("same pair11  ", i , "  ", second_index)
# #                     print("same pair11  ", z1 , "  ", z2)

#                     z1, z2 = digit_indices_one[d][i], digit_indices_zero[d][i - min_index + k]  ###  not same pair  one zero 
#                     pairs += [[x[z1], x[z2]]]
#                     y_diff = get_four_conditions(y[z1], y[z2])
# #                 y_diff = get_two_conditions(y[z1], y[z2])
# #                 y_diff = get_y_xor(y[z1], y[z2])
# #                 y_diff = get_hamming_dis(y[z1], y[z2])
#                     labels += [y_diff] 
#                     ids += [(str(z1) + "_" + str(z2))]
#                     tmp = second_index - min_index
# #                     print("not same pair10  ", i , "  ", tmp)
# #                     print("not same pair10  ", z1 , "  ", z2)
    
#                 else:                                                                 ##  if there are more 0 than 1
# #                     z1, z2 = digit_indices_one[d][i - min_index], digit_indices_one[d][i - min_index + 2]  ###  same pair 
#                     z1, z2 = digit_indices_zero[d][i], digit_indices_zero[d][i + k]     ###  same pair   zero zero 
#                     pairs += [[x[z1], x[z2]]]                     
#                     y_diff = get_four_conditions(y[z1], y[z2])
# #                 y_diff = get_two_conditions(y[z1], y[z2])
# #                 y_diff = get_y_xor(y[z1], y[z2])
# #                 y_diff = get_hamming_dis(y[z1], y[z2])
#                     labels += [y_diff]
#                     ids += [(str(z1) + "_" + str(z2))]
# #                     print("same pair00  ", i , "  ", second_index)
# #                     print("same pair00  ", z1 , "  ", z2)

#                     z1, z2 = digit_indices_one[d][i - min_index + k], digit_indices_zero[d][i]  ###  not same pair one zero
#                     pairs += [[x[z1], x[z2]]]
#                     y_diff = get_four_conditions(y[z1], y[z2])
# #                 y_diff = get_two_conditions(y[z1], y[z2])
# #                 y_diff = get_y_xor(y[z1], y[z2])
# #                 y_diff = get_hamming_dis(y[z1], y[z2])
#                     labels += [y_diff]
#                     ids += [(str(z1) + "_" + str(z2))]
#                     tmp = second_index - min_index
# #                     print("not same pair10 ", tmp , "  ", i)
# #                     print("not same pair10 ", z1 , "  ", z2)
        
#     return np.array(pairs), np.array(labels), ids

In [27]:
def create_pairs(x, y, digit_indices_one, digit_indices_zero, offset):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    ids = []
    
    seen = set()
#     n = min([len(digit_indices[d]) for d in range(num_labels)]) - 1
#     print('n is ', n)
    for d in range(num_labels):
        
#         min_index = min(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0]) - offset
#         max_index = max(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0])
        
        for k in range(1, offset):
            min_index = min(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0]) - k
            max_index = max(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0]) - k
            
            for i in range(min_index):
                second_index = i + k
                
                z1, z2 = digit_indices_one[d][i], digit_indices_one[d][i + k]    ###  same pair  one one 
                seen, exist = checkIfExist(seen, z1, z2)
                if exist == False:
                    pairs += [[x[z1], x[z2]]]
                    y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                    labels += [y_diff]
                    ids += [(str(z1) + "_" + str(z2))]
#                 print("same pair11  ", i , "  ", second_index)
#                 print("same pair11  ", z1 , "  ", z2)
                
                z1, z2 = digit_indices_zero[d][i], digit_indices_zero[d][i + k]  ###  same pair   zero zero 
                seen, exist = checkIfExist(seen, z1, z2)
                if exist == False:
                    pairs += [[x[z1], x[z2]]]                       
                    y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                    labels += [y_diff]
                    ids += [(str(z1) + "_" + str(z2))]
#                 print("same pair00  ", i , "  ", second_index)
#                print("same pair00  ", z1 , "  ", z2)

                z1, z2 = digit_indices_one[d][i], digit_indices_zero[d][i + k]       ###  not same pair  one zero
                seen, exist = checkIfExist(seen, z1, z2)
                if exist == False:
                    pairs += [[x[z1], x[z2]]]
                    y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                    labels += [y_diff]
                    ids += [(str(z1) + "_" + str(z2))]
#                 print("not same pair10  ", i , "  ", second_index)
#                print("not same pair10  ", z1 , "  ", z2)
                
                z1, z2 = digit_indices_one[d][i + k], digit_indices_zero[d][i]  ###  not same pair   zero one 
                seen, exist = checkIfExist(seen, z1, z2)
                if exist == False:
                    pairs += [[x[z1], x[z2]]]
                    y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                    labels += [y_diff]
                    ids += [(str(z1) + "_" + str(z2))]
#                 print("not same pair01  ", second_index , "  ", i)
#                print("not same pair01  ", z1 , "  ", z2)
            
            for i in range(min_index, max_index):
                
                second_index = i + k
                
                if digit_indices_one[d].shape[0] > digit_indices_zero[d].shape[0]:    ##   if there are more 1 than 0 
                    z1, z2 = digit_indices_one[d][i], digit_indices_one[d][i + k]     ###  same pair  one one
                    seen, exist = checkIfExist(seen, z1, z2)
                    if exist == False:
                        pairs += [[x[z1], x[z2]]]                      
                        y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                        labels += [y_diff]
                        ids += [(str(z1) + "_" + str(z2))]
#                     print("same pair11  ", i , "  ", second_index)
#                     print("same pair11  ", z1 , "  ", z2)

                    z1, z2 = digit_indices_one[d][i], digit_indices_zero[d][i - min_index + k]  ###  not same pair  one zero 
                    seen, exist = checkIfExist(seen, z1, z2)
                    if exist == False:
                        pairs += [[x[z1], x[z2]]]
                        y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                        labels += [y_diff] 
                        ids += [(str(z1) + "_" + str(z2))]
                        tmp = second_index - min_index
#                     print("not same pair10  ", i , "  ", tmp)
#                     print("not same pair10  ", z1 , "  ", z2)
    
                else:                                                                 ##  if there are more 0 than 1
#                     z1, z2 = digit_indices_one[d][i - min_index], digit_indices_one[d][i - min_index + 2]  ###  same pair 
                    z1, z2 = digit_indices_zero[d][i], digit_indices_zero[d][i + k]     ###  same pair   zero zero 
                    seen, exist = checkIfExist(seen, z1, z2)
                    if exist == False:
                        pairs += [[x[z1], x[z2]]]                     
                        y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                        labels += [y_diff]
                        ids += [(str(z1) + "_" + str(z2))]
#                     print("same pair00  ", i , "  ", second_index)
#                     print("same pair00  ", z1 , "  ", z2)

                    if ((i - min_index + k) < digit_indices_one[d].shape[0]):
                        z1, z2 = digit_indices_one[d][i - min_index + k], digit_indices_zero[d][i]  ###  not same pair one zero
                        seen, exist = checkIfExist(seen, z1, z2)
                        if exist == False:
                            pairs += [[x[z1], x[z2]]]
                            y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                            labels += [y_diff]
                            ids += [(str(z1) + "_" + str(z2))]
                            tmp = second_index - min_index
#                     print("not same pair10 ", tmp , "  ", i)
#                     print("not same pair10 ", z1 , "  ", z2)
        
    return np.array(pairs), np.array(labels), ids

In [28]:
def get_digit_indices():
    num_train_samples = len(X_train)
    num_test_samples = len(X_test)
    print(num_train_samples)
    print(num_test_samples)

    digit_indices_train_one = []
    digit_indices_train_zero = []
    for i in range(num_labels):
        indexes_for_label_is_one = []
        indexes_for_label_is_zero = []
        for m in range(num_train_samples):
            if Y_train[m][i] == 1:
                indexes_for_label_is_one.append(m)
            elif Y_train[m][i] == 0:
                indexes_for_label_is_zero.append(m)
            else: 
                print("Wrong Label ", Y_train[m][i])
        
        digit_indices_train_one.append(np.asarray(indexes_for_label_is_one))   
        digit_indices_train_zero.append(np.asarray(indexes_for_label_is_zero))  

    digit_indices_test_one = []
    digit_indices_test_zero = []
    for i in range(num_labels):
        indexes_for_label_is_one = []
        indexes_for_label_is_zero = []
        for m in range(num_test_samples):
            if Y_test[m][i] == 1:
                indexes_for_label_is_one.append(m)
            elif Y_test[m][i] == 0:
                indexes_for_label_is_zero.append(m)
            else: 
                print("Wrong Label ", Y_test[m][i])
        digit_indices_test_one.append(np.asarray(indexes_for_label_is_one))   
        digit_indices_test_zero.append(np.asarray(indexes_for_label_is_zero))  

    
    print(len(digit_indices_train_one))
    print(len(digit_indices_train_zero))

    print(len(digit_indices_test_one))
    print(len(digit_indices_test_zero))

    # print(digit_indices_train_one[0].shape)
    # print(digit_indices_train_zero[0].shape)

    # print(digit_indices_train_one[1].shape)
    # print(digit_indices_train_zero[1].shape)
    return digit_indices_train_one, digit_indices_train_zero, digit_indices_test_one, digit_indices_test_zero

In [29]:
def remove_Repeated_file(original_pairs, original_labels, all_ids):
    feature_pairs_fliter = []
    label_filter = []
    
    seen = set()
    uniq = []
#     for string_id in all_ids:
    for i in range(0, len(all_ids)):
        string_id1 = all_ids[i]
        reversed_id = string_id1.split('_')[1] + string_id1.split('_')[0]

        if string_id1 not in seen or reversed_id not in seen:
            uniq.append(string_id1)
    #         print(type(x))
    #         print(string_id)
            seen.add(string_id1)
#             part1 = string_id1.split('_')[0]
#             part2 = string_id1.split('_')[1]
#     #         print("part1 ", part1)
#     #         print("part2 ", part2)
#             reversed_id = part2 + part1
    #         print(reversed_id)
            seen.add(reversed_id)
        
            feature_pairs_fliter.append(original_pairs[i])
            label_filter.append(original_labels[i])
    
    feature_pairs_fliter = np.asarray(feature_pairs_fliter)
    label_filter = np.asarray(label_filter)
    return feature_pairs_fliter, label_filter, uniq

In [30]:
digit_indices_train_one, digit_indices_train_zero, digit_indices_test_one, digit_indices_test_zero = get_digit_indices()

9600
2400
10
10
10
10


In [31]:
# def get_four_conditions(y1, y2):
#     y_diff = np.zeros(num_labels * 3)
#     for i in range(num_labels):
#         if y1[i] == y2[i] and y1[i] == 1:
#             y_diff[i*3 + 0] = 1
#         elif y1[i] == y2[i] and y1[i] == 0:
#             y_diff[i*3 + 1] = 1
#         elif y1[i] != y2[i] and y1[i] == 1:
#             y_diff[i*3 + 2] = 1
#         elif y1[i] != y2[i] and y1[i] == 0:
#             y_diff[i*3 + 2] = 1
#     return y_diff
#
# y1 = [0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
# y2 = [0, 0, 1, 0, 1, 0, 1, 1, 0 ,1]
# y1 = np.asarray(y1)
# y2 = np.asarray(y2)
# y_diff = get_four_conditions(y1, y2)

# for i in range(num_labels):
#     print(y_diff[i*3 + 0])
#     print(y_diff[i*3 + 1])
#     print(y_diff[i*3 + 2])
#     print()

In [32]:
del Normalized_All_Features
del All_Labels_One_Hot
del All_Features
del All_Labels

In [33]:
tr_pairs, tr_y, tr_pairs_ids = create_pairs(X_train, Y_train, digit_indices_train_one, digit_indices_train_zero, 50)
print(tr_pairs.shape)
print(tr_y.shape)
print(len(tr_pairs_ids))

(4172839, 2, 19, 128)
(4172839, 30)
4172839


In [34]:
# tr_pairs_fliter, tr_y_filter, train_id_uniq = remove_Repeated_file(tr_pairs, tr_y, tr_pairs_ids)
# print(tr_pairs_fliter.shape)
# print(tr_y_filter.shape)
# print(len(train_id_uniq))

tr_pairs_fliter = tr_pairs
tr_y_filter = tr_y
train_id_uniq = tr_pairs_ids
del tr_pairs
del tr_y
del tr_pairs_ids

In [35]:
te_pairs, te_y, te_pairs_ids = create_pairs(X_test, Y_test, digit_indices_test_one, digit_indices_test_zero, 50)
print(te_pairs.shape)
print(te_y.shape)
print(len(te_pairs_ids))

(865013, 2, 19, 128)
(865013, 30)
865013


In [36]:
# te_pairs_fliter, te_y_filter, test_id_uniq = remove_Repeated_file(te_pairs, te_y, te_pairs_ids)
# print(te_pairs_fliter.shape)
# print(te_y_filter.shape)
# print(len(test_id_uniq))

te_pairs_fliter = te_pairs
te_y_filter = te_y
test_id_uniq = te_pairs_ids

In [37]:
del te_pairs
del te_y
del te_pairs_ids

In [38]:
# tr_y_filter = np.expand_dims(tr_y_filter, axis=1)
# print(tr_y_filter.shape)

# te_y_filter = np.expand_dims(te_y_filter, axis=1)
# print(te_y_filter.shape)

In [39]:
# num_train_samples = 732913
# num_test_samples = 100421
# num_train_samples = 868083
# num_test_samples = 155811
num_train_samples = 4172839
num_test_samples = 865013

tr_y_filter = tr_y_filter.reshape(num_train_samples, 10, 3)
te_y_filter = te_y_filter.reshape(num_test_samples, 10, 3)

In [40]:
train_out_1 = tr_y_filter[:num_train_samples, 0, :3]
train_out_2 = tr_y_filter[:num_train_samples, 1, :3]
train_out_3 = tr_y_filter[:num_train_samples, 2, :3]
train_out_4 = tr_y_filter[:num_train_samples, 3, :3]
train_out_5 = tr_y_filter[:num_train_samples, 4, :3]
train_out_6 = tr_y_filter[:num_train_samples, 5, :3]
train_out_7 = tr_y_filter[:num_train_samples, 6, :3]
train_out_8 = tr_y_filter[:num_train_samples, 7, :3]
train_out_9 = tr_y_filter[:num_train_samples, 8, :3]
train_out_10 = tr_y_filter[:num_train_samples, 9, :3]

print(train_out_1.shape)
print(train_out_2.shape)
print(train_out_3.shape)
print(train_out_4.shape)
print(train_out_5.shape)
print(train_out_6.shape)
print(train_out_7.shape)
print(train_out_8.shape)
print(train_out_9.shape)
print(train_out_10.shape)

(4172839, 3)
(4172839, 3)
(4172839, 3)
(4172839, 3)
(4172839, 3)
(4172839, 3)
(4172839, 3)
(4172839, 3)
(4172839, 3)
(4172839, 3)


In [41]:
test_out_1 = te_y_filter[:num_test_samples, 0, :3]
test_out_2 = te_y_filter[:num_test_samples, 1, :3]
test_out_3 = te_y_filter[:num_test_samples, 2, :3]
test_out_4 = te_y_filter[:num_test_samples, 3, :3]
test_out_5 = te_y_filter[:num_test_samples, 4, :3]
test_out_6 = te_y_filter[:num_test_samples, 5, :3]
test_out_7 = te_y_filter[:num_test_samples, 6, :3]
test_out_8 = te_y_filter[:num_test_samples, 7, :3]
test_out_9 = te_y_filter[:num_test_samples, 8, :3]
test_out_10 = te_y_filter[:num_test_samples, 9, :3]

print(test_out_1.shape)
print(test_out_2.shape)
print(test_out_3.shape)
print(test_out_4.shape)
print(test_out_5.shape)
print(test_out_6.shape)
print(test_out_7.shape)
print(test_out_8.shape)
print(test_out_9.shape)
print(test_out_10.shape)

(865013, 3)
(865013, 3)
(865013, 3)
(865013, 3)
(865013, 3)
(865013, 3)
(865013, 3)
(865013, 3)
(865013, 3)
(865013, 3)


In [42]:
# train_out_1 = np.expand_dims(train_out_1, axis=1)
# train_out_2 = np.expand_dims(train_out_2, axis=1)
# train_out_3 = np.expand_dims(train_out_3, axis=1)
# train_out_4 = np.expand_dims(train_out_4, axis=1)
# train_out_5 = np.expand_dims(train_out_5, axis=1)
# train_out_6 = np.expand_dims(train_out_6, axis=1)
# train_out_7 = np.expand_dims(train_out_7, axis=1)
# train_out_8 = np.expand_dims(train_out_8, axis=1)
# train_out_9 = np.expand_dims(train_out_9, axis=1)
# train_out_10 = np.expand_dims(train_out_10, axis=1)

# test_out_1 = np.expand_dims(test_out_1, axis=1)
# test_out_2 = np.expand_dims(test_out_2, axis=1)
# test_out_3 = np.expand_dims(test_out_3, axis=1)
# test_out_4 = np.expand_dims(test_out_4, axis=1)
# test_out_5 = np.expand_dims(test_out_5, axis=1)
# test_out_6 = np.expand_dims(test_out_6, axis=1)
# test_out_7 = np.expand_dims(test_out_7, axis=1)
# test_out_8 = np.expand_dims(test_out_8, axis=1)
# test_out_9 = np.expand_dims(test_out_9, axis=1)
# test_out_10 = np.expand_dims(test_out_10, axis=1)


In [43]:
print(train_out_10.shape)
print(test_out_10.shape)

(4172839, 3)
(865013, 3)


In [44]:
# network definition
base_network = create_base_network()

input_a = Input(shape=(time_steps, freq_bins))
input_b = Input(shape=(time_steps, freq_bins))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

### trying to add attention module before the comparision but not sure whether it will work, since the Siamese Network Will o longer be actual Siamese
# attention_a2 = Dense(hidden_units)(processed_a)
# attention_a2 = BatchNormalization()(attention_a2)
# attention_a2 = Activation('relu')(attention_a2)
# attention_a2 = Dropout(drop_rate)(attention_a2)

# attention_b2 = Dense(hidden_units)(processed_b)
# attention_b2 = BatchNormalization()(attention_b2)
# attention_b2 = Activation('relu')(attention_b2)
# attention_b2 = Dropout(drop_rate)(attention_b2)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [45]:
def attention_pooling(inputs, **kwargs):
    [out, att] = inputs

    epsilon = 1e-7
    att = K.clip(att, epsilon, 1. - epsilon)
    normalized_att = att / K.sum(att, axis=1)[:, None, :]

    return K.sum(out * normalized_att, axis=1)

In [46]:
def pooling_shape(input_shape):

    if isinstance(input_shape, list):
        (sample_num, time_steps, freq_bins) = input_shape[0]

    else:
        (sample_num, time_steps, freq_bins) = input_shape

    return (sample_num, freq_bins)

In [47]:
# distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])


###### This is not actually distance (it didn't sum up each element) so that the output layer is not 1 node 
# L1_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
# L1_distance = L1_layer([processed_a, processed_b])

ConcatLayer = keras.layers.concatenate([processed_a, processed_b], axis=-1)

# L2_layer = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)
# L2_distance = L2_layer([processed_a, processed_b])

# prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(L2_distance)

# Cousin_layer = Lambda(cosine_distance, output_shape=cos_dist_output_shape)
# Cousin_distance = Cousin_layer([processed_a, processed_b])

# print(Cousin_distance.output_shape)
# print(K.shape(Cousin_distance))

# model = Model(inputs = [input_a, input_b], outputs = L1_distance)
# model = Model(inputs = [input_a, input_b], outputs = [L2_distance, L2_distance])
# model = Model(inputs = [input_a, input_b], outputs = [Cousin_distance])

# model = Model(inputs = [input_a, input_b], outputs = prediction)

# model.summary()
# print()
# base_network.summary()

# fc1 = Dense(512, kernel_initializer="glorot_uniform")(distance)
# fc1 = Dropout(0.2)(fc1)
# fc1 = Activation("relu")(fc1)


# fc1 = Dense(256, activation='relu')(Cousin_distance)
# fc1 = Dense(256, activation='relu')(L2_distance)
# fc1 = Dense(256)(L1_distance)
fc1 = Dense(256)(ConcatLayer)
fc1 = BatchNormalization()(fc1)
fc1 = Dropout(0.2)(fc1)
fc1 = Activation("relu")(fc1)

fc2 = Dense(128)(fc1)
fc2 = BatchNormalization()(fc2)
fc2 = Activation('relu')(fc2)

# fc2 = Conv2D(8, (1, 4), strides=(1, 2), padding='same')(fc1)
# fc2 = Reshape((19, -1))(fc2)
# #     model.add(Permute((2, 1)))
# #     a5 = LSTM(128, return_sequences=True)(a1)
# fc2 = Dense(128, activation='relu')(fc2)

cla_1 =  Dense(3, activation='softmax')(fc2)
att_1 = Dense(3, activation='softmax')(fc2)
output_1 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_1, att_1])

cla_2 =  Dense(3, activation='softmax')(fc2)
att_2 = Dense(3, activation='softmax')(fc2)
output_2 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_2, att_2])

cla_3 =  Dense(3, activation='softmax')(fc2)
att_3 = Dense(3, activation='softmax')(fc2)
output_3 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_3, att_3])

cla_4 =  Dense(3, activation='softmax')(fc2)
att_4 = Dense(3, activation='softmax')(fc2)
output_4 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_4, att_4])

cla_5 =  Dense(3, activation='softmax')(fc2)
att_5 = Dense(3, activation='softmax')(fc2)
output_5 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_5, att_5])

cla_6 =  Dense(3, activation='softmax')(fc2)
att_6 = Dense(3, activation='softmax')(fc2)
output_6 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_6, att_6])

cla_7 =  Dense(3, activation='softmax')(fc2)
att_7 = Dense(3, activation='softmax')(fc2)
output_7 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_7, att_7])

cla_8 =  Dense(3, activation='softmax')(fc2)
att_8 = Dense(3, activation='softmax')(fc2)
output_8 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_8, att_8])

cla_9 =  Dense(3, activation='softmax')(fc2)
att_9 = Dense(3, activation='softmax')(fc2)
output_9 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_9, att_9])

cla_10 =  Dense(3, activation='softmax')(fc2)
att_10 = Dense(3, activation='softmax')(fc2)
output_10 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_10, att_10])

model = Model(inputs=[input_a, input_b], outputs=[output_1, output_2, output_3, output_4,
                                                  output_5, output_6, output_7, output_8, output_9, output_10])


###########
# fc2 = MaxPooling1D(pool_length=19, stride=None, border_mode='valid')(fc2)

# output_1 =  Dense(3, activation='softmax')(fc2)
# # att_1 = Dense(3, activation='softmax')(fc2)
# # output_1 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_1, att_1])

# output_2 =  Dense(3, activation='softmax')(fc2)
# # att_2 = Dense(3, activation='softmax')(fc2)
# # output_2 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_2, att_2])

# output_3 =  Dense(3, activation='softmax')(fc2)
# # att_3 = Dense(3, activation='softmax')(fc2)
# # output_3 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_3, att_3])

# output_4 =  Dense(3, activation='softmax')(fc2)
# # att_4 = Dense(3, activation='softmax')(fc2)
# # output_4 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_4, att_4])

# output_5 =  Dense(3, activation='softmax')(fc2)
# # att_5 = Dense(3, activation='softmax')(fc2)
# # output_5 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_5, att_5])

# output_6 =  Dense(3, activation='softmax')(fc2)
# # att_6 = Dense(3, activation='softmax')(fc2)
# # output_6 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_6, att_6])

# output_7 =  Dense(3, activation='softmax')(fc2)
# # att_7 = Dense(3, activation='softmax')(fc2)
# # output_7 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_7, att_7])

# output_8 =  Dense(3, activation='softmax')(fc2)
# # att_8 = Dense(3, activation='softmax')(fc2)
# # output_8 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_8, att_8])

# output_9 =  Dense(3, activation='softmax')(fc2)
# # att_9 = Dense(3, activation='softmax')(fc2)
# # output_9 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_9, att_9])

# output_10 =  Dense(3, activation='softmax')(fc2)
# # att_10 = Dense(3, activation='softmax')(fc2)
# # output_10 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_10, att_10])

# model = Model(inputs=[input_a, input_b], outputs=[output_1, output_2, output_3, output_4,
#                                                   output_5, output_6, output_7, output_8, output_9, output_10])

###################

# cla_1 =  Dense(64, activation='relu')(fc2)
# att_1 = Dense(64, activation='softmax')(fc2)
# fc3_1 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_1, att_1])

# cla_2 =  Dense(64, activation='relu')(fc2)
# att_2 = Dense(64, activation='softmax')(fc2)
# fc3_2 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_2, att_2])

# cla_3 =  Dense(64, activation='relu')(fc2)
# att_3 = Dense(64, activation='softmax')(fc2)
# fc3_3 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_3, att_3])

# cla_4 =  Dense(64, activation='relu')(fc2)
# att_4 = Dense(64, activation='softmax')(fc2)
# fc3_4 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_4, att_4])

# cla_5 =  Dense(64, activation='relu')(fc2)
# att_5 = Dense(64, activation='softmax')(fc2)
# fc3_5 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_5, att_5])

# cla_6 =  Dense(64, activation='relu')(fc2)
# att_6 = Dense(64, activation='softmax')(fc2)
# fc3_6 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_6, att_6])

# cla_7 =  Dense(64, activation='relu')(fc2)
# att_7 = Dense(64, activation='softmax')(fc2)
# fc3_7 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_7, att_7])

# cla_8 =  Dense(64, activation='relu')(fc2)
# att_8 = Dense(64, activation='softmax')(fc2)
# fc3_8 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_8, att_8])

# cla_9 =  Dense(64, activation='relu')(fc2)
# att_9 = Dense(64, activation='softmax')(fc2)
# fc3_9 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_9, att_9])

# cla_10 =  Dense(64, activation='relu')(fc2)
# att_10 = Dense(64, activation='softmax')(fc2)
# fc3_10 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_10, att_10])

# output_1 = Dense(3, activation = 'softmax')(fc3_1)
# output_2 = Dense(3, activation = 'softmax')(fc3_2)
# output_3 = Dense(3, activation = 'softmax')(fc3_3)
# output_4 = Dense(3, activation = 'softmax')(fc3_4)
# output_5 = Dense(3, activation = 'softmax')(fc3_5)
# output_6 = Dense(3, activation = 'softmax')(fc3_6)
# output_7 = Dense(3, activation = 'softmax')(fc3_7)
# output_8 = Dense(3, activation = 'softmax')(fc3_8)
# output_9 = Dense(3, activation = 'softmax')(fc3_9)
# output_10 = Dense(3, activation = 'softmax')(fc3_10)

# model = Model(inputs=[input_a, input_b], outputs=[output_1, output_2, output_3, output_4,output_5, 
#                                                   output_6, output_7, output_8, output_9, output_10])
    

# pred = Dense(3, kernel_initializer="glorot_uniform")(fc2)
# pred = Activation("sigmoid")(pred)

#### use maxpooling to pool the time domain (28)
# pred = MaxPooling1D(pool_length=28, stride=None, border_mode='valid')(pred)

# pred = keras.layers.concatenate([output_1, output_2, output_3, output_4, 
#                                  output_5, output_6, output_7, output_8, output_9, output_10], axis=-1)


# model = Model(inputs=[input_a, input_b], outputs=[output_1, output_2, output_3, output_4,
#                                                   output_5, output_6, output_7, output_8, output_9, output_10])

model.summary()
print()
base_network.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 19, 128)      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 19, 128)      0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 19, 128)      67584       input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 19, 256)      0           model_1[1][0]                    
          

In [48]:
cos_dist_output_shape

<function __main__.cos_dist_output_shape(shapes)>

In [49]:
# te_y_1 = np.expand_dims(te_y, axis=1)
# test_y = np.concatenate((te_y_1, te_y_1, te_y_1, te_y_1, te_y_1, te_y_1, te_y_1, te_y_1, te_y_1, te_y_1), axis=1)
# print(test_y.shape)

# tr_y_1 = np.expand_dims(tr_y, axis=1)
# train_y = np.concatenate((tr_y_1, tr_y_1, tr_y_1, tr_y_1, tr_y_1, tr_y_1, tr_y_1, tr_y_1, tr_y_1, tr_y_1), axis=1)
# print(train_y.shape)

In [50]:
# train_y = np.expand_dims(train_y, axis=1)
# print(train_y.shape)

# test_y = np.expand_dims(test_y, axis=1)
# print(test_y.shape)

In [51]:
# train_out_1 = np.expand_dims(train_out_1, axis=1)
# train_out_2 = np.expand_dims(train_out_2, axis=1)
# train_out_3 = np.expand_dims(train_out_3, axis=1)
# train_out_4 = np.expand_dims(train_out_4, axis=1)
# train_out_5 = np.expand_dims(train_out_5, axis=1)
# train_out_6 = np.expand_dims(train_out_6, axis=1)
# train_out_7 = np.expand_dims(train_out_7, axis=1)
# train_out_8 = np.expand_dims(train_out_8, axis=1)
# train_out_9 = np.expand_dims(train_out_9, axis=1)
# train_out_10 = np.expand_dims(train_out_10, axis=1)


# test_out_1 = np.expand_dims(test_out_1, axis=1)
# test_out_2 = np.expand_dims(test_out_2, axis=1)
# test_out_3 = np.expand_dims(test_out_3, axis=1)
# test_out_4 = np.expand_dims(test_out_4, axis=1)
# test_out_5 = np.expand_dims(test_out_5, axis=1)
# test_out_6 = np.expand_dims(test_out_6, axis=1)
# test_out_7 = np.expand_dims(test_out_7, axis=1)
# test_out_8 = np.expand_dims(test_out_8, axis=1)
# test_out_9 = np.expand_dims(test_out_9, axis=1)
# test_out_10 = np.expand_dims(test_out_10, axis=1)


In [52]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
#         print("true_positives ",K.eval(true_positives))
#         print("possible_positives ",K.eval(possible_positives))
#         print("recall         ",K.eval(recall))
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        
#         print("true_positives ",K.eval(true_positives))
#         print("predicted_positives ",K.eval(predicted_positives))
#         print("precision      ",K.eval(precision))
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    f_score = 2*((precision*recall)/(precision+recall+K.epsilon()))
    
#     print("precision     ",K.eval(precision))
#     print("recall        ",K.eval(recall))
#     print("f_score       ",K.eval(f_score))
    return f_score

In [53]:
from keras.callbacks import Callback

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [54]:
callbacks = [
    EarlyStoppingByLossVal(monitor='val_loss', value=5, verbose=1),
    # EarlyStopping(monitor='val_loss', patience=2, verbose=0),
#     ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0),
]

In [55]:
epochs = 3

In [56]:
# train
rms = RMSprop()
# model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
# model.compile(loss=contrastive_loss, optimizer=rms)
# model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'], optimizer=rms)

multiple_losses = ['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy',
                  'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy']
# model.compile(loss= multiple_losses, metrics=['binary_accuracy', metrics.categorical_accuracy], optimizer=rms)
# model.compile(loss= multiple_losses, metrics=[metrics.categorical_accuracy], optimizer=rms)
model.compile(loss= multiple_losses, metrics=[metrics.categorical_accuracy, f1_m,precision_m, recall_m], optimizer=rms)


# history = model.fit([tr_pairs_fliter[:, 0], tr_pairs_fliter[:, 1]], [tr_y_filter],
#           batch_size=128,
#           epochs=epochs,
#           validation_data=([te_pairs_fliter[:, 0], te_pairs_fliter[:, 1]], [te_y_filter]))

history = model.fit([tr_pairs_fliter[:, 0], tr_pairs_fliter[:, 1]], [train_out_1,
                                                                    train_out_2,
                                                                    train_out_3,
                                                                    train_out_4,
                                                                    train_out_5,
                                                                    train_out_6,
                                                                    train_out_7,
                                                                    train_out_8,
                                                                    train_out_9,
                                                                    train_out_10],
          batch_size=128,
          epochs=epochs,
          validation_data=([te_pairs_fliter[:, 0], te_pairs_fliter[:, 1]], [test_out_1,
                                                                           test_out_2,
                                                                           test_out_3,
                                                                           test_out_4,
                                                                           test_out_5,
                                                                           test_out_6,
                                                                           test_out_7,
                                                                           test_out_8,
                                                                           test_out_9,
                                                                           test_out_10]))


# plt.figure(figsize=(8, 4))
# plt.subplot(1, 2, 1)
# plot_train_history(history, 'loss', 'val_loss')
# plt.subplot(1, 2, 2)
# plot_train_history(history, 'accuracy', 'val_accuracy')
# plt.show()

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 4172839 samples, validate on 865013 samples
Epoch 1/3
4172839/4172839 [==============================] - 2006s 481us/step - loss: 6.2280 - lambda_1_loss: 0.7032 - lambda_2_loss: 0.6986 - lambda_3_loss: 0.5844 - lambda_4_loss: 0.6233 - lambda_5_loss: 0.6247 - lambda_6_loss: 0.6422 - lambda_7_loss: 0.6278 - lambda_8_loss: 0.5919 - lambda_9_loss: 0.5283 - lambda_10_loss: 0.6037 - lambda_1_categorical_accuracy: 0.6847 - lambda_1_f1_m: 0.3744 - lambda_1_precision_m: 0.7959 - lambda_1_recall_m: 0.2469 - lambda_2_categorical_accuracy: 0.6948 - lambda_2_f1_m: 0.4794 - lambda_2_precision_m: 0.7531 - lambda_2_recall_m: 0.3530 - lambda_3_categorical_accuracy: 0.7534 - lambda_3_f1_m: 0.4219 - lambda_3_precision_m: 0.8622 - lambda_3_recall_m: 0.2812 - lambda_4_categorical_accuracy: 0.7348 - lambda_4_f1_m: 0.4734 - lambda_4_precision_m: 0.7863 - lambda_4_recall_m: 0.

4172839/4172839 [==============================] - 1963s 470us/step - loss: 5.8173 - lambda_1_loss: 0.6520 - lambda_2_loss: 0.6562 - lambda_3_loss: 0.5442 - lambda_4_loss: 0.5912 - lambda_5_loss: 0.5839 - lambda_6_loss: 0.5958 - lambda_7_loss: 0.5943 - lambda_8_loss: 0.5425 - lambda_9_loss: 0.4913 - lambda_10_loss: 0.5657 - lambda_1_categorical_accuracy: 0.7152 - lambda_1_f1_m: 0.3743 - lambda_1_precision_m: 0.8350 - lambda_1_recall_m: 0.2421 - lambda_2_categorical_accuracy: 0.7203 - lambda_2_f1_m: 0.4959 - lambda_2_precision_m: 0.7808 - lambda_2_recall_m: 0.3642 - lambda_3_categorical_accuracy: 0.7741 - lambda_3_f1_m: 0.4163 - lambda_3_precision_m: 0.8880 - lambda_3_recall_m: 0.2728 - lambda_4_categorical_accuracy: 0.7509 - lambda_4_f1_m: 0.4484 - lambda_4_precision_m: 0.8170 - lambda_4_recall_m: 0.3100 - lambda_5_categorical_accuracy: 0.7578 - lambda_5_f1_m: 0.5533 - lambda_5_precision_m: 0.8177 - lambda_5_recall_m: 0.4192 - lambda_6_categorical_accuracy: 0.7445 - lambda_6_f1_m: 0.38

In [ ]:
history

In [ ]:
te_pairs_fliter.shape
input_1 = te_pairs_fliter[:num_test_samples, 0, :19, :128]
input_2 = te_pairs_fliter[:num_test_samples, 1, :19, :128]

In [ ]:
input_1.shape

In [ ]:
print(model.test_on_batch([input_1, input_2], [test_out_1, test_out_2,test_out_3, test_out_4,test_out_5,
                                               test_out_6,test_out_7,test_out_8,test_out_9,test_out_10]))

In [ ]:
# test = model.predict_on_batch([input_1, input_2])

In [ ]:
test_out_1.shape

In [ ]:
Performance_tmp = model.test_on_batch([input_1, input_2], [test_out_1, test_out_2,test_out_3, test_out_4,test_out_5,
                                               test_out_6,test_out_7,test_out_8,test_out_9,test_out_10])

In [ ]:
print(len(Performance_tmp))
Performance = Performance_tmp[11:]
print(len(Performance))
# print(Performance)
acc_summary = []
f1_summary = []
precision_summary = []
recall_summary = []
for i in range(0, 10):
#     print(i * 4 + 0)
#     print(Performance[i * 4 + 0])
    acc_summary.append(Performance[i * 4 + 0])
    f1_summary.append(Performance[i * 4 + 1])
    precision_summary.append(Performance[i * 4 + 2])
    recall_summary.append(Performance[i * 4 + 3])

In [ ]:
acc_summary

In [ ]:
f1_summary

In [ ]:
precision_summary

In [ ]:
recall_summary

In [ ]:
Database_X = X_train
Database_Y = Y_train
size_of_database = len(Database_Y) 

In [ ]:
def indicateor_function(predicted_dic, groundtruth_list, Threshold):
    index = predicted_dic.get("index")
    for i in range(len(groundtruth_list)):
        if(groundtruth_list[i].get("index") == index):
            actual_similarity = groundtruth_list[i].get("similarity")
            break
    if(actual_similarity >= Threshold):
#         print(" actual_similarity  ", actual_similarity, "  True")
        return True
    elif (actual_similarity < Threshold):
#         print(" actual_similarity  ", actual_similarity, "  False")
        return False
    else: 
#         print("Wrong ")
        return False

In [ ]:
def Average_Precision_at_Top_K(ranked_list, groundtruth_list, Top_K, Threshold):
    
    average_precision = 0 
    num_positive_hits = 0
    for i in range(0, Top_K):
#     for i in range(size_of_database - Top_K, size_of_database):
        if(indicateor_function(ranked_list[i], groundtruth_list, Threshold) == True):
            num_positive_hits = num_positive_hits + 1
            average_precision = average_precision + num_positive_hits/(i+1)

    if (num_positive_hits != 0):
        average_precision = average_precision/num_positive_hits
        
    return average_precision

In [ ]:
def Baseline_Average_Precision_at_Top_K(groundtruth_list, Top_K, Threshold):
    
    average_precision = 0 
    num_positive_hits = 0
    
    random_top_k = random.sample(groundtruth_list, Top_K)
    
    for i in range(0, Top_K):
        actual_similarity = random_top_k[i].get("similarity")
        if(actual_similarity >= Threshold):
            num_positive_hits = num_positive_hits + 1
            average_precision = average_precision + num_positive_hits/(i+1)
    if (num_positive_hits != 0):
        average_precision = average_precision/num_positive_hits    
        
    return average_precision

In [ ]:
num_labels = 10 

def Compute_Mean_Average_Precision(Threshold):
    
    map_1 = 0
    map_5 = 0
    map_10 = 0
    map_30 = 0
    map_50 = 0
    map_100 = 0
    map_database_size = 0
    
    base_map_1 = 0
    base_map_5 = 0
    base_map_10 = 0
    base_map_30 = 0
    base_map_50 = 0
    base_map_100 = 0
    base_map_database_size = 0

    for i in range (0, len(X_test)):
        
        print("Query ", i)
    
        predicted_ranked_list = []
        groundtruth_list = []
        for k in range(len(Database_X)):
#         print(X_test[i].shape)
            input_1 = np.expand_dims(X_test[i], axis=0)
            input_2 = np.expand_dims(Database_X[k], axis=0)
            pair = [input_1, input_2]
            output = model.predict(pair)
        
            ## get one hot presentation of predicted results 
            prediced_results = []
            for n in range (0, num_labels):
                predicted = np.zeros(3)
                index = np.argmax(output[n])
                predicted[index] = 1
                prediced_results.append(predicted)
        
            ### compute the level of similarity based on predicted result
            predicted_similarity = 0
            for n in range (0, num_labels):
                predicted_similarity = predicted_similarity + prediced_results[n][0] + prediced_results[n][1]
#         print("predict similarity " , predicted_similarity)
            thisdict = {"index": k, "similarity": predicted_similarity}
            predicted_ranked_list.append(thisdict)
            
            ### compute the level of similarity of of groundtruth annotation 
            ground_truth = get_four_conditions(Y_test[i], Database_Y[k])
            ground_truth = ground_truth.reshape(num_labels, 3)
        
#         print("Query ", i, "  ", "Database ", k)
#         print("predicted ")
#         print(np.array(prediced_results))
#         print("ground_tr ")
#         print(ground_truth)  
#         print()
        
            actual_similarity = 0
            for n in range (0, num_labels):
                actual_similarity = actual_similarity + ground_truth[n][0] + ground_truth[n][1]
#         print("ground_truth similarity " , actual_similarity) 
            thisdict = {"index": k, "similarity": actual_similarity}
            groundtruth_list.append(thisdict)
        
#         print("Query ", i)
#     predicted_ranked_list = RankBasedonSimilarity(predicted_ranked_list)
        Ranked_List = sorted(predicted_ranked_list, key = lambda i: i['similarity'], reverse=True)
    
        
        
        average_precision_1 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 1, Threshold)
        map_1 = map_1 + average_precision_1
        
        average_precision_5 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 5, Threshold)
        map_5 = map_5 + average_precision_5
        
        average_precision_10 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 10, Threshold)
        map_10 = map_10 + average_precision_10
        
        average_precision_30 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 30, Threshold)
        map_30 = map_30 + average_precision_30
        
        average_precision_50 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 50, Threshold)
        map_50 = map_50 + average_precision_50
        
        average_precision_100 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 100, Threshold)
        map_100 = map_100 + average_precision_100
        
#         average_precision_database_size = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, size_of_database, Threshold)
#         map_database_size = map_database_size + average_precision_database_size
        
        
        base_average_precision_1 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 1, Threshold)
        base_map_1 = base_map_1 + base_average_precision_1
        
        base_average_precision_5 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 5, Threshold)
        base_map_5 = base_map_5 + base_average_precision_5
        
        base_average_precision_10 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 10, Threshold)
        base_map_10 = base_map_10 + base_average_precision_10
        
        base_average_precision_30 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 30, Threshold)
        base_map_30 = base_map_30 + base_average_precision_30
        
        base_average_precision_50 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 50, Threshold)
        base_map_50 = base_map_50 + base_average_precision_50
        
        base_average_precision_100 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 100, Threshold)
        base_map_100 = base_map_100 + base_average_precision_100
        
#         base_average_precision_database_size = Baseline_Average_Precision_at_Top_K(groundtruth_list, size_of_database, Threshold)
#         base_map_database_size = base_map_database_size + base_average_precision_database_size
        
#         print("hello")
#         print(base_map_10)
        
        
    map_1 = map_1/len(X_test)
    map_5 = map_5/len(X_test) 
    map_10 = map_10/len(X_test) 
    map_30 = map_30/len(X_test)
    map_50 = map_50/len(X_test) 
    map_100 = map_100/len(X_test) 
#     map_database_size = map_database_size/len(X_test) 
    

    base_map_1 = base_map_1/len(X_test)
    base_map_5 = base_map_5/len(X_test) 
    base_map_10 = base_map_10/len(X_test) 
    base_map_30 = base_map_30/len(X_test)
    base_map_50 = base_map_50/len(X_test) 
    base_map_100 = base_map_100/len(X_test) 
#     base_map_database_size = base_map_database_size/len(X_test) 
    
#     map_results = [map_1, map_5, map_10, map_30, map_50, map_100, map_database_size]
#     base_results = [base_map_1, base_map_5, base_map_10, base_map_30, base_map_50, base_map_100, base_map_database_size]
    
    map_results = [map_1, map_5, map_10, map_30, map_50, map_100]
    base_results = [base_map_1, base_map_5, base_map_10, base_map_30, base_map_50, base_map_100]
    
    return map_results, base_results

In [57]:
def save_model(model, Save_Model_Dir):
#     Save_Model_Dir = Dir + "DCASE/saved_models/"
    # serialize model to JSON
    model_json = model.to_json()
    
    json_dir = Save_Model_Dir + "model.json"
    with open(json_dir, "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    h5_dir = Save_Model_Dir + "model.h5"
    model.save_weights(h5_dir)
    print("Saved model to disk")

In [58]:
Save_Model_Dir = "/mnt/tmp/SAVED_MODELS/Synthesized_Concat_20190916/"
save_model(model, Save_Model_Dir)

Saved model to disk


In [73]:
MAP10, B_MAP10 = Compute_Mean_Average_Precision(10)
print("10")
print(MAP10)
print(B_MAP10)

MAP9, B_MAP9 = Compute_Mean_Average_Precision(9)
print("9")
print(MAP9)
print(B_MAP9)

MAP8, B_MAP8 = Compute_Mean_Average_Precision(8)
print("8")
print(MAP8)
print(B_MAP8)

# MAP7, B_MAP7 = Compute_Mean_Average_Precision(7)
# print("7")
# print(MAP7)
# print(B_MAP7)

# MAP6, B_MAP6 = Compute_Mean_Average_Precision(6)
# print("6")
# print(MAP6)
# print(B_MAP76)

Query  0
Query  1
Query  2
Query  3
Query  4
Query  5
Query  6
Query  7
Query  8
Query  9
Query  10
Query  11
Query  12
Query  13
Query  14
Query  15
Query  16
Query  17
Query  18
Query  19
Query  20
Query  21
Query  22
Query  23
Query  24
Query  25
Query  26
Query  27
Query  28
Query  29
Query  30
Query  31
Query  32
Query  33
Query  34
Query  35
Query  36
Query  37
Query  38
Query  39
Query  40
Query  41
Query  42
Query  43
Query  44
Query  45
Query  46
Query  47
Query  48
Query  49
Query  50
Query  51
Query  52
Query  53
Query  54
Query  55
Query  56
Query  57
Query  58
Query  59
Query  60
Query  61
Query  62
Query  63
Query  64
Query  65
Query  66
Query  67
Query  68
Query  69
Query  70
Query  71
Query  72
Query  73
Query  74
Query  75
Query  76
Query  77
Query  78
Query  79
Query  80
Query  81
Query  82
Query  83
Query  84
Query  85
Query  86
Query  87
Query  88
Query  89
Query  90
Query  91
Query  92
Query  93
Query  94
Query  95
Query  96
Query  97
Query  98
Query  99
Query  100

Query  755
Query  756
Query  757
Query  758
Query  759
Query  760
Query  761
Query  762
Query  763
Query  764
Query  765
Query  766
Query  767
Query  768
Query  769
Query  770
Query  771
Query  772
Query  773
Query  774
Query  775
Query  776
Query  777
Query  778
Query  779
Query  780
Query  781
Query  782
Query  783
Query  784
Query  785
Query  786
Query  787
Query  788
Query  789
Query  790
Query  791
Query  792
Query  793
Query  794
Query  795
Query  796
Query  797
Query  798
Query  799
Query  800
Query  801
Query  802
Query  803
Query  804
Query  805
Query  806
Query  807
Query  808
Query  809
Query  810
Query  811
Query  812
Query  813
Query  814
Query  815
Query  816
Query  817
Query  818
Query  819
Query  820
Query  821
Query  822
Query  823
Query  824
Query  825
Query  826
Query  827
Query  828
Query  829
Query  830
Query  831
Query  832
Query  833
Query  834
Query  835
Query  836
Query  837
Query  838
Query  839
Query  840
Query  841
Query  842
Query  843
Query  844
Query  845

Query  1459
Query  1460
Query  1461
Query  1462
Query  1463
Query  1464
Query  1465
Query  1466
Query  1467
Query  1468
Query  1469
Query  1470
Query  1471
Query  1472
Query  1473
Query  1474
Query  1475
Query  1476
Query  1477
Query  1478
Query  1479
Query  1480
Query  1481
Query  1482
Query  1483
Query  1484
Query  1485
Query  1486
Query  1487
Query  1488
Query  1489
Query  1490
Query  1491
Query  1492
Query  1493
Query  1494
Query  1495
Query  1496
Query  1497
Query  1498
Query  1499
Query  1500
Query  1501
Query  1502
Query  1503
Query  1504
Query  1505
Query  1506
Query  1507
Query  1508
Query  1509
Query  1510
Query  1511
Query  1512
Query  1513
Query  1514
Query  1515
Query  1516
Query  1517
Query  1518
Query  1519
Query  1520
Query  1521
Query  1522
Query  1523
Query  1524
Query  1525
Query  1526
Query  1527
Query  1528
Query  1529
Query  1530
Query  1531
Query  1532
Query  1533
Query  1534
Query  1535
Query  1536
Query  1537
Query  1538
Query  1539
Query  1540
Query  1541
Quer

Query  2142
Query  2143
Query  2144
Query  2145
Query  2146
Query  2147
Query  2148
Query  2149
Query  2150
Query  2151
Query  2152
Query  2153
Query  2154
Query  2155
Query  2156
Query  2157
Query  2158
Query  2159
Query  2160
Query  2161
Query  2162
Query  2163
Query  2164
Query  2165
Query  2166
Query  2167
Query  2168
Query  2169
Query  2170
Query  2171
Query  2172
Query  2173
Query  2174
Query  2175
Query  2176
Query  2177
Query  2178
Query  2179
Query  2180
Query  2181
Query  2182
Query  2183
Query  2184
Query  2185
Query  2186
Query  2187
Query  2188
Query  2189
Query  2190
Query  2191
Query  2192
Query  2193
Query  2194
Query  2195
Query  2196
Query  2197
Query  2198
Query  2199
Query  2200
Query  2201
Query  2202
Query  2203
Query  2204
Query  2205
Query  2206
Query  2207
Query  2208
Query  2209
Query  2210
Query  2211
Query  2212
Query  2213
Query  2214
Query  2215
Query  2216
Query  2217
Query  2218
Query  2219
Query  2220
Query  2221
Query  2222
Query  2223
Query  2224
Quer

Query  451
Query  452
Query  453
Query  454
Query  455
Query  456
Query  457
Query  458
Query  459
Query  460
Query  461
Query  462
Query  463
Query  464
Query  465
Query  466
Query  467
Query  468
Query  469
Query  470
Query  471
Query  472
Query  473
Query  474
Query  475
Query  476
Query  477
Query  478
Query  479
Query  480
Query  481
Query  482
Query  483
Query  484
Query  485
Query  486
Query  487
Query  488
Query  489
Query  490
Query  491
Query  492
Query  493
Query  494
Query  495
Query  496
Query  497
Query  498
Query  499
Query  500
Query  501
Query  502
Query  503
Query  504
Query  505
Query  506
Query  507
Query  508
Query  509
Query  510
Query  511
Query  512
Query  513
Query  514
Query  515
Query  516
Query  517
Query  518
Query  519
Query  520
Query  521
Query  522
Query  523
Query  524
Query  525
Query  526
Query  527
Query  528
Query  529
Query  530
Query  531
Query  532
Query  533
Query  534
Query  535
Query  536
Query  537
Query  538
Query  539
Query  540
Query  541

...

Query  1868
Query  1869
Query  1870
Query  1871
Query  1872
Query  1873
Query  1874
Query  1875
Query  1876
Query  1877
Query  1878
Query  1879
Query  1880
Query  1881
Query  1882
Query  1883
Query  1884
Query  1885
Query  1886
Query  1887
Query  1888
Query  1889
Query  1890
Query  1891
Query  1892
Query  1893
Query  1894
Query  1895
Query  1896
Query  1897
Query  1898
Query  1899
Query  1900
Query  1901
Query  1902
Query  1903
Query  1904
Query  1905
Query  1906
Query  1907
Query  1908
Query  1909
Query  1910
Query  1911
Query  1912
Query  1913
Query  1914
Query  1915
Query  1916
Query  1917
Query  1918
Query  1919
Query  1920
Query  1921
Query  1922
Query  1923
Query  1924
Query  1925
Query  1926
Query  1927
Query  1928
Query  1929
Query  1930
Query  1931
Query  1932
Query  1933
Query  1934
Query  1935
Query  1936
Query  1937
Query  1938
Query  1939
Query  1940
Query  1941
Query  1942
Query  1943
Query  1944
Query  1945
Query  1946
Query  1947
Query  1948
Query  1949
Query  1950
Quer

Query  152
Query  153
Query  154
Query  155
Query  156
Query  157
Query  158
Query  159
Query  160
Query  161
Query  162
Query  163
Query  164
Query  165
Query  166
Query  167
Query  168
Query  169
Query  170
Query  171
Query  172
Query  173
Query  174
Query  175
Query  176
Query  177
Query  178
Query  179
Query  180
Query  181
Query  182
Query  183
Query  184
Query  185
Query  186
Query  187
Query  188
Query  189
Query  190
Query  191
Query  192
Query  193
Query  194
Query  195
Query  196
Query  197
Query  198
Query  199
Query  200
Query  201
Query  202
Query  203
Query  204
Query  205
Query  206
Query  207
Query  208
Query  209
Query  210
Query  211
Query  212
Query  213
Query  214
Query  215
Query  216
Query  217
Query  218
Query  219
Query  220
Query  221
Query  222
Query  223
Query  224
Query  225
Query  226
Query  227
Query  228
Query  229
Query  230
Query  231
Query  232
Query  233
Query  234
Query  235
Query  236
Query  237
Query  238
Query  239
Query  240
Query  241
Query  242

Query  897
Query  898
Query  899
Query  900
Query  901
Query  902
Query  903
Query  904
Query  905
Query  906
Query  907
Query  908
Query  909
Query  910
Query  911
Query  912
Query  913
Query  914
Query  915
Query  916
Query  917
Query  918
Query  919
Query  920
Query  921
Query  922
Query  923
Query  924
Query  925
Query  926
Query  927
Query  928
Query  929
Query  930
Query  931
Query  932
Query  933
Query  934
Query  935
Query  936
Query  937
Query  938
Query  939
Query  940
Query  941
Query  942
Query  943
Query  944
Query  945
Query  946
Query  947
Query  948
Query  949
Query  950
Query  951
Query  952
Query  953
Query  954
Query  955
Query  956
Query  957
Query  958
Query  959
Query  960
Query  961
Query  962
Query  963
Query  964
Query  965
Query  966
Query  967
Query  968
Query  969
Query  970
Query  971
Query  972
Query  973
Query  974
Query  975
Query  976
Query  977
Query  978
Query  979
Query  980
Query  981
Query  982
Query  983
Query  984
Query  985
Query  986
Query  987

Query  1589
Query  1590
Query  1591
Query  1592
Query  1593
Query  1594
Query  1595
Query  1596
Query  1597
Query  1598
Query  1599
Query  1600
Query  1601
Query  1602
Query  1603
Query  1604
Query  1605
Query  1606
Query  1607
Query  1608
Query  1609
Query  1610
Query  1611
Query  1612
Query  1613
Query  1614
Query  1615
Query  1616
Query  1617
Query  1618
Query  1619
Query  1620
Query  1621
Query  1622
Query  1623
Query  1624
Query  1625
Query  1626
Query  1627
Query  1628
Query  1629
Query  1630
Query  1631
Query  1632
Query  1633
Query  1634
Query  1635
Query  1636
Query  1637
Query  1638
Query  1639
Query  1640
Query  1641
Query  1642
Query  1643
Query  1644
Query  1645
Query  1646
Query  1647
Query  1648
Query  1649
Query  1650
Query  1651
Query  1652
Query  1653
Query  1654
Query  1655
Query  1656
Query  1657
Query  1658
Query  1659
Query  1660
Query  1661
Query  1662
Query  1663
Query  1664
Query  1665
Query  1666
Query  1667
Query  1668
Query  1669
Query  1670
Query  1671
Quer

Query  2272
Query  2273
Query  2274
Query  2275
Query  2276
Query  2277
Query  2278
Query  2279
Query  2280
Query  2281
Query  2282
Query  2283
Query  2284
Query  2285
Query  2286
Query  2287
Query  2288
Query  2289
Query  2290
Query  2291
Query  2292
Query  2293
Query  2294
Query  2295
Query  2296
Query  2297
Query  2298
Query  2299
Query  2300
Query  2301
Query  2302
Query  2303
Query  2304
Query  2305
Query  2306
Query  2307
Query  2308
Query  2309
Query  2310
Query  2311
Query  2312
Query  2313
Query  2314
Query  2315
Query  2316
Query  2317
Query  2318
Query  2319
Query  2320
Query  2321
Query  2322
Query  2323
Query  2324
Query  2325
Query  2326
Query  2327
Query  2328
Query  2329
Query  2330
Query  2331
Query  2332
Query  2333
Query  2334
Query  2335
Query  2336
Query  2337
Query  2338
Query  2339
Query  2340
Query  2341
Query  2342
Query  2343
Query  2344
Query  2345
Query  2346
Query  2347
Query  2348
Query  2349
Query  2350
Query  2351
Query  2352
Query  2353
Query  2354
Quer

In [ ]:
# MAP10, B_MAP10 = Compute_Mean_Average_Precision(10)
# print("10")
# print(MAP10)
# print(B_MAP10)

# MAP9, B_MAP9 = Compute_Mean_Average_Precision(9)
# print("9")
# print(MAP9)
# print(B_MAP9)

# MAP8, B_MAP8 = Compute_Mean_Average_Precision(8)
# print("8")
# print(MAP8)
# print(B_MAP8)

# MAP7, B_MAP7 = Compute_Mean_Average_Precision(7)
# print("7")
# print(MAP7)
# print(B_MAP7)

# MAP6, B_MAP6 = Compute_Mean_Average_Precision(6)
# print("6")
# print(MAP6)
# print(B_MAP76)

Query  0
Query  1
Query  2
Query  3
Query  4
Query  5
Query  6
Query  7
Query  8
Query  9
Query  10
Query  11
Query  12
Query  13
Query  14
Query  15
Query  16
Query  17
Query  18
Query  19
Query  20
Query  21
Query  22
Query  23
Query  24
Query  25
Query  26
Query  27
Query  28
Query  29
Query  30
Query  31
Query  32
Query  33
Query  34
Query  35
Query  36
Query  37
Query  38
Query  39
Query  40
Query  41
Query  42
Query  43
Query  44
Query  45
Query  46
Query  47
Query  48
Query  49
Query  50
Query  51
Query  52
Query  53
Query  54
Query  55
Query  56
Query  57
Query  58
Query  59
Query  60
Query  61
Query  62
Query  63
Query  64
Query  65
Query  66
Query  67
Query  68
Query  69
Query  70
Query  71
Query  72
Query  73
Query  74
Query  75
Query  76
Query  77
Query  78
Query  79
Query  80
Query  81
Query  82
Query  83
Query  84
Query  85
Query  86
Query  87
Query  88
Query  89
Query  90
Query  91
Query  92
Query  93
Query  94
Query  95
Query  96
Query  97
Query  98
Query  99
Query  100

Query  755
Query  756
Query  757
Query  758
Query  759
Query  760
Query  761
Query  762
Query  763
Query  764
Query  765
Query  766
Query  767
Query  768
Query  769
Query  770
Query  771
Query  772
Query  773
Query  774
Query  775
Query  776
Query  777
Query  778
Query  779
Query  780
Query  781
Query  782
Query  783
Query  784
Query  785
Query  786
Query  787
Query  788
Query  789
Query  790
Query  791
Query  792
Query  793
Query  794
Query  795
Query  796
Query  797
Query  798
Query  799
Query  800
Query  801
Query  802
Query  803
Query  804
Query  805
Query  806
Query  807
Query  808
Query  809
Query  810
Query  811
Query  812
Query  813
Query  814
Query  815
Query  816
Query  817
Query  818
Query  819
Query  820
Query  821
Query  822
Query  823
Query  824
Query  825
Query  826
Query  827
Query  828
Query  829
Query  830
Query  831
Query  832
Query  833
Query  834
Query  835
Query  836
Query  837
Query  838
Query  839
Query  840
Query  841
Query  842
Query  843
Query  844
Query  845

Query  1459
Query  1460
Query  1461
Query  1462
Query  1463
Query  1464
Query  1465
Query  1466
Query  1467
Query  1468
Query  1469
Query  1470
Query  1471
Query  1472
Query  1473
Query  1474
Query  1475
Query  1476
Query  1477
Query  1478
Query  1479
Query  1480
Query  1481
Query  1482
Query  1483
Query  1484
Query  1485
Query  1486
Query  1487
Query  1488
Query  1489
Query  1490
Query  1491
Query  1492
Query  1493
Query  1494
Query  1495
Query  1496
Query  1497
Query  1498
Query  1499
Query  1500
Query  1501
Query  1502
Query  1503
Query  1504
Query  1505
Query  1506
Query  1507
Query  1508
Query  1509
Query  1510
Query  1511
Query  1512
Query  1513
Query  1514
Query  1515
Query  1516
Query  1517
Query  1518
Query  1519
Query  1520
Query  1521
Query  1522
Query  1523
Query  1524
Query  1525
Query  1526
Query  1527
Query  1528
Query  1529
Query  1530
Query  1531
Query  1532
Query  1533
Query  1534
Query  1535
Query  1536
Query  1537
Query  1538
Query  1539
Query  1540
Query  1541
Quer

Query  2142
Query  2143
Query  2144
Query  2145
Query  2146
Query  2147
Query  2148
Query  2149
Query  2150
Query  2151
Query  2152
Query  2153
Query  2154
Query  2155
Query  2156
Query  2157
Query  2158
Query  2159
Query  2160
Query  2161
Query  2162
Query  2163
Query  2164
Query  2165
Query  2166
Query  2167
Query  2168
Query  2169
Query  2170
Query  2171
Query  2172
Query  2173
Query  2174
Query  2175
Query  2176
Query  2177
Query  2178
Query  2179
Query  2180
Query  2181
Query  2182
Query  2183
Query  2184
Query  2185
Query  2186
Query  2187
Query  2188
Query  2189
Query  2190
Query  2191
Query  2192
Query  2193
Query  2194
Query  2195
Query  2196
Query  2197
Query  2198
Query  2199
Query  2200
Query  2201
Query  2202
Query  2203
Query  2204
Query  2205
Query  2206
Query  2207
Query  2208
Query  2209
Query  2210
Query  2211
Query  2212
Query  2213
Query  2214
Query  2215
Query  2216
Query  2217
Query  2218
Query  2219
Query  2220
Query  2221
Query  2222
Query  2223
Query  2224
Quer

Query  446
Query  447
Query  448
Query  449
Query  450
Query  451
Query  452
Query  453
Query  454
Query  455
Query  456
Query  457
Query  458
Query  459
Query  460
Query  461
Query  462
Query  463
Query  464
Query  465
Query  466
Query  467
Query  468
Query  469
Query  470
Query  471
Query  472
Query  473
Query  474
Query  475
Query  476
Query  477
Query  478
Query  479
Query  480
Query  481
Query  482
Query  483
Query  484
Query  485
Query  486
Query  487
Query  488
Query  489
Query  490
Query  491
Query  492
Query  493
Query  494
Query  495
Query  496
Query  497
Query  498
Query  499
Query  500
Query  501
Query  502
Query  503
Query  504
Query  505
Query  506
Query  507
Query  508
Query  509
Query  510
Query  511
Query  512
Query  513
Query  514
Query  515
Query  516
Query  517
Query  518
Query  519
Query  520
Query  521
Query  522
Query  523
Query  524
Query  525
Query  526
Query  527
Query  528
Query  529
Query  530
Query  531
Query  532
Query  533
Query  534
Query  535
Query  536

Query  1175
Query  1176
Query  1177
Query  1178
Query  1179
Query  1180
Query  1181
Query  1182
Query  1183
Query  1184
Query  1185
Query  1186
Query  1187
Query  1188
Query  1189
Query  1190
Query  1191
Query  1192
Query  1193
Query  1194
Query  1195
Query  1196
Query  1197
Query  1198
Query  1199
Query  1200
Query  1201
Query  1202
Query  1203
Query  1204
Query  1205
Query  1206
Query  1207
Query  1208
Query  1209
Query  1210
Query  1211
Query  1212
Query  1213
Query  1214
Query  1215
Query  1216
Query  1217
Query  1218
Query  1219
Query  1220
Query  1221
Query  1222
Query  1223
Query  1224
Query  1225
Query  1226
Query  1227
Query  1228
Query  1229
Query  1230
Query  1231
Query  1232
Query  1233
Query  1234
Query  1235
Query  1236
Query  1237
Query  1238
Query  1239
Query  1240
Query  1241
Query  1242
Query  1243
Query  1244
Query  1245
Query  1246
Query  1247
Query  1248
Query  1249
Query  1250
Query  1251
Query  1252
Query  1253
Query  1254
Query  1255
Query  1256
Query  1257
Quer

In [ ]:
num_samples = test_out_1.shape[0]

def get_Precision_Recall_F1(test_out, pred_out):
    number = num_test_samples * 3
#     test_out_one_dimension = test_out.reshape(number)
#     pred_out_one_dimension = pred_out.reshape(number)

    test_out_one_dimension = test_out
    pred_out_one_dimension = pred_out

    true_label = K.variable(test_out_one_dimension)
    pred_label = K.variable(pred_out_one_dimension)

    precision = precision_m(true_label, pred_label)
    recall = recall_m(true_label, pred_label)
    f_score = f1_m(true_label, pred_label)
#     print(score)
#     print(K.eval(score))
    return precision, recall, f_score
    

def FirstCase(test_out):
    pred_out = np.zeros((num_samples, 3))
    for i in range(num_samples):
        pred_out[i][0] = 1
    # pred_out_1 = np.ones((100421, 3))
    binary_acc = K.mean(K.equal(test_out, K.round(pred_out)))
    categorical_acc = K.mean(K.equal(K.argmax(test_out, axis=-1), K.argmax(pred_out, axis=-1)))
    precision, recall, f_score = get_Precision_Recall_F1(test_out, pred_out)
#     print(K.eval(binary_acc))
#     print(K.eval(categorical_acc))
    return categorical_acc, precision, recall, f_score

def SecondCase(test_out):
    pred_out = np.zeros((num_samples, 3))
    for i in range(num_samples):
        pred_out[i][1] = 1
    # pred_out_1 = np.ones((100421, 3))
    binary_acc = K.mean(K.equal(test_out, K.round(pred_out)))
    categorical_acc = K.mean(K.equal(K.argmax(test_out, axis=-1), K.argmax(pred_out, axis=-1)))
    precision, recall, f_score = get_Precision_Recall_F1(test_out, pred_out)
#     print(K.eval(binary_acc))
#     print(K.eval(categorical_acc))
    return categorical_acc, precision, recall, f_score

def ThirdCase(test_out):
    pred_out = np.zeros((num_samples, 3))
    for i in range(num_samples):
        pred_out[i][2] = 1
    # pred_out_1 = np.ones((100421, 3))
    binary_acc = K.mean(K.equal(test_out, K.round(pred_out)))
    categorical_acc = K.mean(K.equal(K.argmax(test_out, axis=-1), K.argmax(pred_out, axis=-1)))
    precision, recall, f_score = get_Precision_Recall_F1(test_out, pred_out)
#     print(K.eval(binary_acc))
#     print(K.eval(categorical_acc))
    return categorical_acc, precision, recall, f_score


print('1111111')
baseline1 = []
baseline1_precision = []
baseline1_recall = []
baseline1_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_1)
# print(K.eval(categorical_acc),"  ", K.eval(precision),"  ", K.eval(recall),"  ", K.eval(f1))
baseline1.append(K.eval(categorical_acc))
baseline1_precision.append(K.eval(precision))
baseline1_recall.append(K.eval(recall))
baseline1_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_1)
baseline1.append(K.eval(categorical_acc))
baseline1_precision.append(K.eval(precision))
baseline1_recall.append(K.eval(recall))
baseline1_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_1)
baseline1.append(K.eval(categorical_acc))
baseline1_precision.append(K.eval(precision))
baseline1_recall.append(K.eval(recall))
baseline1_f1.append(K.eval(f1))


print('2222222')
baseline2 = []
baseline2_precision = []
baseline2_recall = []
baseline2_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_2)
baseline2.append(K.eval(categorical_acc))
baseline2_precision.append(K.eval(precision))
baseline2_recall.append(K.eval(recall))
baseline2_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_2)
baseline2.append(K.eval(categorical_acc))
baseline2_precision.append(K.eval(precision))
baseline2_recall.append(K.eval(recall))
baseline2_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_2)
baseline2.append(K.eval(categorical_acc))
baseline2_precision.append(K.eval(precision))
baseline2_recall.append(K.eval(recall))
baseline2_f1.append(K.eval(f1))
print()



print('3333333')
baseline3 = []
baseline3_precision = []
baseline3_recall = []
baseline3_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_3)
baseline3.append(K.eval(categorical_acc))
baseline3_precision.append(K.eval(precision))
baseline3_recall.append(K.eval(recall))
baseline3_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_3)
baseline3.append(K.eval(categorical_acc))
baseline3_precision.append(K.eval(precision))
baseline3_recall.append(K.eval(recall))
baseline3_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_3)
baseline3.append(K.eval(categorical_acc))
baseline3_precision.append(K.eval(precision))
baseline3_recall.append(K.eval(recall))
baseline3_f1.append(K.eval(f1))
print()


print('4444444')
baseline4 = []
baseline4_precision = []
baseline4_recall = []
baseline4_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_4)
baseline4.append(K.eval(categorical_acc))
baseline4_precision.append(K.eval(precision))
baseline4_recall.append(K.eval(recall))
baseline4_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_4)
baseline4.append(K.eval(categorical_acc))
baseline4_precision.append(K.eval(precision))
baseline4_recall.append(K.eval(recall))
baseline4_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_4)
baseline4.append(K.eval(categorical_acc))
baseline4_precision.append(K.eval(precision))
baseline4_recall.append(K.eval(recall))
baseline4_f1.append(K.eval(f1))
print()


print('5555555')
baseline5 = []
baseline5_precision = []
baseline5_recall = []
baseline5_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_5)
baseline5.append(K.eval(categorical_acc))
baseline5_precision.append(K.eval(precision))
baseline5_recall.append(K.eval(recall))
baseline5_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_5)
baseline5.append(K.eval(categorical_acc))
baseline5_precision.append(K.eval(precision))
baseline5_recall.append(K.eval(recall))
baseline5_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_5)
baseline5.append(K.eval(categorical_acc))
baseline5_precision.append(K.eval(precision))
baseline5_recall.append(K.eval(recall))
baseline5_f1.append(K.eval(f1))
print()


print('6666666')
baseline6 = []
baseline6_precision = []
baseline6_recall = []
baseline6_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_6)
# print(categorical_acc,"  ", precision,"  ", recall,"  ", f1)
baseline6.append(K.eval(categorical_acc))
baseline6_precision.append(K.eval(precision))
baseline6_recall.append(K.eval(recall))
baseline6_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_6)
baseline6.append(K.eval(categorical_acc))
baseline6_precision.append(K.eval(precision))
baseline6_recall.append(K.eval(recall))
baseline6_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_6)
baseline6.append(K.eval(categorical_acc))
baseline6_precision.append(K.eval(precision))
baseline6_recall.append(K.eval(recall))
baseline6_f1.append(K.eval(f1))
print()

print('7777777')
baseline7 = []
baseline7_precision = []
baseline7_recall = []
baseline7_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_7)
baseline7.append(K.eval(categorical_acc))
baseline7_precision.append(K.eval(precision))
baseline7_recall.append(K.eval(recall))
baseline7_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_7)
baseline7.append(K.eval(categorical_acc))
baseline7_precision.append(K.eval(precision))
baseline7_recall.append(K.eval(recall))
baseline7_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_7)
baseline7.append(K.eval(categorical_acc))
baseline7_precision.append(K.eval(precision))
baseline7_recall.append(K.eval(recall))
baseline7_f1.append(K.eval(f1))
print()

print('8888888')
baseline8 = []
baseline8_precision = []
baseline8_recall = []
baseline8_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_8)
baseline8.append(K.eval(categorical_acc))
baseline8_precision.append(K.eval(precision))
baseline8_recall.append(K.eval(recall))
baseline8_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_8)
baseline8.append(K.eval(categorical_acc))
baseline8_precision.append(K.eval(precision))
baseline8_recall.append(K.eval(recall))
baseline8_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_8)
baseline8.append(K.eval(categorical_acc))
baseline8_precision.append(K.eval(precision))
baseline8_recall.append(K.eval(recall))
baseline8_f1.append(K.eval(f1))
print()

In [ ]:
print('9999999')
baseline9 = []
baseline9_precision = []
baseline9_recall = []
baseline9_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_9)
baseline9.append(K.eval(categorical_acc))
baseline9_precision.append(K.eval(precision))
baseline9_recall.append(K.eval(recall))
baseline9_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_9)
baseline9.append(K.eval(categorical_acc))
baseline9_precision.append(K.eval(precision))
baseline9_recall.append(K.eval(recall))
baseline9_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_9)
baseline9.append(K.eval(categorical_acc))
baseline9_precision.append(K.eval(precision))
baseline9_recall.append(K.eval(recall))
baseline9_f1.append(K.eval(f1))
print()

print('10101010101010')
baseline10 = []
baseline10_precision = []
baseline10_recall = []
baseline10_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_10)
baseline10.append(K.eval(categorical_acc))
baseline10_precision.append(K.eval(precision))
baseline10_recall.append(K.eval(recall))
baseline10_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_10)
baseline10.append(K.eval(categorical_acc))
baseline10_precision.append(K.eval(precision))
baseline10_recall.append(K.eval(recall))
baseline10_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_10)
baseline10.append(K.eval(categorical_acc))
baseline10_precision.append(K.eval(precision))
baseline10_recall.append(K.eval(recall))
baseline10_f1.append(K.eval(f1))
print()

In [ ]:
print(max(baseline1))
print(max(baseline2))
print(max(baseline3))
print(max(baseline4))
print(max(baseline5))
print(max(baseline6))
print(max(baseline7))
print(max(baseline8))
print(max(baseline9))
print(max(baseline10))
print()

print(max(baseline1_precision))
print(max(baseline2_precision))
print(max(baseline3_precision))
print(max(baseline4_precision))
print(max(baseline5_precision))
print(max(baseline6_precision))
print(max(baseline7_precision))
print(max(baseline8_precision))
print(max(baseline9_precision))
print(max(baseline10_precision))
print()

print(max(baseline1_recall))
print(max(baseline2_recall))
print(max(baseline3_recall))
print(max(baseline4_recall))
print(max(baseline5_recall))
print(max(baseline6_recall))
print(max(baseline7_recall))
print(max(baseline8_recall))
print(max(baseline9_recall))
print(max(baseline10_recall))
print()

print(max(baseline1_f1))
print(max(baseline2_f1))
print(max(baseline3_f1))
print(max(baseline4_f1))
print(max(baseline5_f1))
print(max(baseline6_f1))
print(max(baseline7_f1))
print(max(baseline8_f1))
print(max(baseline9_f1))
print(max(baseline10_f1))

In [69]:
print(test[0][0], " " , test_out_1[0])
print(test[0][1], " " , test_out_2[0])
print(test[0][2], " " , test_out_3[0])
print(test[0][3], " " , test_out_4[0])
print(test[0][4], " " , test_out_5[0])
print(test[0][5], " " , test_out_6[0])
print(test[0][6], " " , test_out_7[0])
print(test[0][7], " " , test_out_8[0])
print(test[0][8], " " , test_out_9[0])
print(test[0][9], " " , test_out_10[0])

NameError: name 'test' is not defined

In [ ]:
for i in range(0, 10):
    print(test[i][0], " " , test_out_1[i])
    print(test[i][1], " " , test_out_2[i])
    print(test[i][2], " " , test_out_3[i])
    print(test[i][3], " " , test_out_4[i])
    print(test[i][4], " " , test_out_5[i])
    print(test[i][5], " " , test_out_6[i])
    print(test[i][6], " " , test_out_7[i])
    print(test[i][7], " " , test_out_8[i])
    print(test[i][8], " " , test_out_9[i])
    print(test[i][9], " " , test_out_10[i])
    print()

In [ ]:
print(test_out_1.shape)
print(test_out_2.shape)
print(test_out_3.shape)
print(test_out_4.shape)
print(test_out_5.shape)
print(test_out_6.shape)
print(test_out_7.shape)
print(test_out_8.shape)
print(test_out_9.shape)
print(test_out_10.shape)

In [ ]:
import random
# random.uniform(0, 1)

num_samples = num_test_samples

pred_out_1 = np.random.rand(num_samples, 3)
binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
print()

pred_out_1 = np.ones((num_samples, 3))
binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
print()

In [ ]:
print('11111111')
baseline1 = []
pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline1.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline1.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline1.append(K.eval(categorical_acc))
print()



print('2222222')
baseline2 = []
pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_2, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_2, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline2.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_2, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_2, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline2.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_2, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_2, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline2.append(K.eval(categorical_acc))
print()


print('3333333')
baseline3 = []
pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_3, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_3, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline3.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_3, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_3, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline3.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_3, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_3, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline3.append(K.eval(categorical_acc))
print()


print('444444')
baseline4 = []
pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_4, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_4, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline4.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_4, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_4, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline4.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_4, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_4, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline4.append(K.eval(categorical_acc))
print()


print('55555555')
baseline5 = []
pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_5, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_5, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline5.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_5, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_5, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline5.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_5, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_5, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline5.append(K.eval(categorical_acc))
print()


print('666666')
baseline6 = []
pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_6, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_6, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline6.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_6, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_6, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline6.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_6, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_6, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline6.append(K.eval(categorical_acc))
print()

print('777777')
baseline7 = []
pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_7, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_7, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline7.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_7, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_7, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline7.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_7, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_7, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline7.append(K.eval(categorical_acc))
print()


print('888888')
baseline8 = []
pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_8, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_8, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline8.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_8, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_8, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline8.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_8, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_8, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline8.append(K.eval(categorical_acc))
print()



print('9999999')
baseline9 = []
pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_9, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_9, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline9.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_9, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_9, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline9.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_9, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_9, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline9.append(K.eval(categorical_acc))
print()


print('101010101010')
baseline10 = []
pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][0] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_10, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_10, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline10.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][1] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_10, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_10, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline10.append(K.eval(categorical_acc))
print()

pred_out_1 = np.zeros((num_samples, 3))
for i in range(num_samples):
    pred_out_1[i][2] = 1
# pred_out_1 = np.ones((100421, 3))
binary_acc = K.mean(K.equal(test_out_10, K.round(pred_out_1)))
categorical_acc = K.mean(K.equal(K.argmax(test_out_10, axis=-1), K.argmax(pred_out_1, axis=-1)))
print(K.eval(binary_acc))
print(K.eval(categorical_acc))
baseline10.append(K.eval(categorical_acc))
print()

In [ ]:
print(max(baseline1))
print(max(baseline2))
print(max(baseline3))
print(max(baseline4))
print(max(baseline5))
print(max(baseline6))
print(max(baseline7))
print(max(baseline8))
print(max(baseline9))
print(max(baseline10))

In [ ]:
layer_name = 'model_3'
# layer_name = 'lambda_2'
layer_output = model.get_layer(layer_name).get_output_at(-1)
intermediate_layer_model = Model(inputs = model.input, outputs = layer_output)
embedding_left_sample = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(embedding_left_sample.shape)
print(embedding_left_sample[0].shape)
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# layer_output = get_3rd_layer_output([x])[0]

In [ ]:
embedding_left_sample[100,:,1]

In [ ]:
embedding_left_sample[100,:,2]

In [ ]:
embedding_left_sample[101,:,1]

In [ ]:
layer_name = 'model_1'
# layer_name = 'lambda_2'
layer_output = model.get_layer(layer_name).get_output_at(-2)
intermediate_layer_model = Model(inputs = model.input, outputs = layer_output)
embedding_right_sample = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(embedding_right_sample.shape)
print(embedding_right_sample[0].shape)

In [ ]:
# layer_name = 'sequential_2'
layer_name = 'lambda_1'
intermediate_layer_model = Model(inputs = model.input, outputs = model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(intermediate_output.shape)
print(intermediate_output[0].shape)
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# layer_output = get_3rd_layer_output([x])[0]

In [ ]:
# layer_name = 'sequential_2'
layer_name = 'dense_19'
intermediate_layer_model = Model(inputs = model.input, outputs = model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(intermediate_output.shape)
print(intermediate_output[0].shape)
print(intermediate_output)
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# layer_output = get_3rd_layer_output([x])[0]

In [ ]:
# layer_name = 'sequential_2'
layer_name = 'dense_17'
intermediate_layer_model = Model(inputs = model.input, outputs = model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(intermediate_output.shape)intermediate_output
print(intermediate_output[0].shape)
print(intermediate_output)
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# layer_output = get_3rd_layer_output([x])[0]

In [ ]:
print(intermediate_output[0,:,1].shape)
print(intermediate_output[0,:,2].shape)
print(intermediate_output[0,:,1])
print(intermediate_output[0,:,2])

In [ ]:
# layer_name = 'sequential_2'
layer_name = 'dense_7'
intermediate_layer_model = Model(inputs = model.input, outputs = model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(intermediate_output.shape)
print(intermediate_output[0].shape)
print(intermediate_output)
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# layer_output = get_3rd_layer_output([x])[0]

In [ ]:
embedding_left_sample[0]

In [ ]:
embedding_right_sample[0]

In [ ]:
intermediate_output[0]

In [ ]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    
    pred = y_pred.ravel() > 0.5     ## if I use sigmoid and cross entrpy, then I won't need to make the conversion. It is 
                                    ## always the correct order 
    
#     for i in range(0, len(pred)):
#         print(str(y_true[i]) + '  ' + str(y_pred[i]) + "      " +str(y_true[i]) + '  ' + str(pred[i]))
    return np.mean(pred == y_true)

In [ ]:
# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred[:, 0])
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred[:, 0])

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

# print(y_pred)
# print(te_y)
# for i in range(0, len(y_pred)):
#     print(str(y_pred[i]) + '  ' + str(te_y[i]))

In [ ]:
for i in range (0, len(y_pred)):
    print(str(y_pred[i][0]) + '  ' + str(tr_y[i]))

In [ ]:
tr_y

In [ ]:
# def cos_distance(vects):
#     y_true, y_pred = vects
#     y_true = K.l2_normalize(y_true, axis=-1)
#     y_pred = K.l2_normalize(y_pred, axis=-1)
#     return K.mean(1 - K.sum((y_true * y_pred), axis=-1))

# def cosine_distance(vests):
#     x, y = vests
#     x = K.l2_normalize(x, axis=-1)
#     y = K.l2_normalize(y, axis=-1)
#     return -K.mean(x * y, axis=-1)

# def cos_dist_output_shape(shapes):
#     shape1, shape2 = shapes
#     shape=list(shape1)
#     assert len(shape) == 2
#     shape[-1] = 1
#     return tuple(shape)

# output_cos = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([processed_a, processed_b])